In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Not running on Google Colab")

try:
    from env_vars import *
except:
    print("did not find env_vars.py")
    intra_test_path = 'drive/MyDrive/Intra/test/'
    intra_train_path = 'drive/MyDrive/Intra/train/'

Not running on Google Colab


In [5]:
#merge all files into one dataset
import h5py
import os
import numpy as np

dir_path_train = intra_train_path 
dir_path_test = intra_test_path
# loop through directory and get all matrices

def get_dataset_name(file_name_with_dir):
    file_name_without_dir = file_name_with_dir.split('/')[-1]
    temp = file_name_without_dir.split('_')[:-1]
    dataset_name = "_".join(temp)
    return dataset_name

def get_label(filename):
    if 'rest' in filename:
        label = 0
    elif 'math' in filename:
        label = 1
    elif 'memory' in filename:
        label = 2
    elif 'motor' in filename:
        label = 3
    return label

def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, 'r') as f:
                dataset_name = get_dataset_name(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)


    return dataset, labels



In [6]:
dataset_and_labels_intra = get_all_matrices(dir_path_train)
intra_x_train = np.stack(np.array(dataset_and_labels_intra[0]))
intra_y_train = dataset_and_labels_intra[1]

In [7]:
dataset_and_labels_intra = get_all_matrices(dir_path_test)
intra_x_test = np.stack(np.array(dataset_and_labels_intra[0]))
intra_y_test = dataset_and_labels_intra[1]

In [8]:
intra_x_train.shape, intra_x_test.shape

((32, 248, 35624), (8, 248, 35624))

In [9]:
from tensorflow.keras.utils import to_categorical
 # one hot encode y
intra_y_train = to_categorical(intra_y_train)
intra_y_test = to_categorical(intra_y_test)


## **FFT**

In [10]:
def dataset_fft(dataset):
  window_size = 900
  overlap = 450

  num_subjects = dataset.shape[0]
  num_sensors = dataset.shape[1]
  num_time_steps = dataset.shape[2]
  fft_dataset = []

  for i in range(num_subjects):
    num_windows = num_time_steps // window_size
    spectral_power_results = np.zeros((num_sensors, num_windows, window_size // 2 + 1), dtype=np.float32)
    print('subject: ', i)
    for j in range(num_sensors):
      for k in range(num_windows):
        start_idx = k * overlap
        end_idx = (k + 1) * window_size

        windowed_signal = dataset[i, j, start_idx:end_idx]
        windowed_fft = np.fft.fft(windowed_signal, axis=0)

        spectral_power_results[j, k, :] = np.abs(windowed_fft[:window_size // 2 + 1]) ** 2

    fft_dataset.append(spectral_power_results)
  return np.array(fft_dataset)

In [ ]:
intra_x_train_fft = dataset_fft(intra_x_train)
intra_x_test_fft = dataset_fft(intra_x_test)

subject:  0
subject:  1
subject:  2
subject:  3
subject:  4
subject:  5
subject:  6
subject:  7
subject:  8
subject:  9
subject:  10
subject:  11
subject:  12
subject:  13
subject:  14
subject:  15
subject:  16
subject:  17
subject:  18
subject:  19
subject:  20
subject:  21
subject:  22
subject:  23
subject:  24
subject:  25
subject:  26
subject:  27
subject:  28
subject:  29
subject:  30
subject:  31
subject:  0
subject:  1
subject:  2
subject:  3
subject:  4
subject:  5
subject:  6
subject:  7


In [ ]:
#reshaped_x_train = intra_x_train_fft.transpose(0, 2, 1, 3).reshape(intra_x_train_fft.shape[0], intra_x_train_fft.shape[2], -1)
reshaped_x_train = intra_x_train_fft.reshape((32, 248, -1))
reshaped_x_test = intra_x_test_fft.reshape((8, 248, -1))

In [ ]:
#reshaped_x_test = intra_x_test_fft.transpose(0, 2, 1, 3).reshape(intra_x_test_fft.shape[0], intra_x_test_fft.shape[2], -1)

In [ ]:
reshaped_x_test.shape

(8, 248, 17589)

In [ ]:
reshaped_x_train[0][0][0]

1.9206653e-19

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scaler(matrix):
  scaler = MinMaxScaler()
  result = []
  for subject in matrix:
    temp = scaler.fit_transform(subject)
    result.append(temp)
  return np.array(result)
x
scaled_reshaped_x_train = scaler(reshaped_x_train)
scaled_reshaped_x_test = scaler(reshaped_x_test)

In [ ]:
scaled_reshaped_x_train.shape

(32, 248, 17589)

# Simple LSTM

In [ ]:
NUM_SENSORS = reshaped_x_train.shape[1]
NUM_FEATURES = reshaped_x_train.shape[2]
NUM_OUTPUTS = intra_y_train.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
verbose, epochs, batch_size = 1, 15, 64
model = Sequential()
model.add(LSTM(100, input_shape=(NUM_SENSORS, NUM_FEATURES)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(NUM_OUTPUTS, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit network
model.fit(scaled_reshaped_x_train, intra_y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
 # evaluate model
_, accuracy = model.evaluate(scaled_reshaped_x_test, intra_y_test, batch_size=batch_size, verbose=0)

Epoch 1/15
1/1 [==============================] - 8s 8s/step - loss: 1.3863 - accuracy: 0.2500
Epoch 2/15
1/1 [==============================] - 5s 5s/step - loss: 1.3860 - accuracy: 0.2812
Epoch 3/15
1/1 [==============================] - 5s 5s/step - loss: 1.3863 - accuracy: 0.2812
Epoch 4/15
1/1 [==============================] - 5s 5s/step - loss: 1.3861 - accuracy: 0.2500
Epoch 5/15
1/1 [==============================] - 5s 5s/step - loss: 1.3865 - accuracy: 0.3125
Epoch 6/15
1/1 [==============================] - 5s 5s/step - loss: 1.3867 - accuracy: 0.2812
Epoch 7/15
1/1 [==============================] - 4s 4s/step - loss: 1.3859 - accuracy: 0.2812
Epoch 8/15
1/1 [==============================] - 5s 5s/step - loss: 1.3856 - accuracy: 0.3438
Epoch 9/15
1/1 [==============================] - 4s 4s/step - loss: 1.3874 - accuracy: 0.1562
Epoch 10/15
1/1 [==============================] - 5s 5s/step - loss: 1.3870 - accuracy: 0.2812
Epoch 11/15
1/1 [==============================] 